# 动态给实例／class 绑定属性和方法

正常情况下，当我们定义了一个 class，创建了一个 class的实例后，我们可以**给该实例绑定任何属性和方法**，这就是**动态语言的灵活性**。

下面是给一个实例绑定属性和方法，**注意给某个实例绑定方法，对另一个实例是不起作用的**

为了给所有实例绑定方法，可以给 class绑定方法。

In [11]:
# 定义 class
class Student():
    pass

s = Student()

# 动态地给实例绑定属性
s.name = 'Jamie'
print(s.name)

# 动态地给实例绑定方法
def set_age(self,age): #定义一个函数作为实例的方法
    self.age = age 
    
from types import MethodType
s.set_age = MethodType(set_age,s) # 给实例绑定方法
s.set_age(28)
print(s.age)

# 该方法对另一个实例不起作用
s2 =Student()
s2.set_age(30) # 报错 AttributeError: 'student' object has no attribute 'set_age'

# 动态给class绑定方法
def set_score(self,score):
    self.score = score
    
Student.set_score = set_score
s.set_score(100)
s2.set_score (99)
print(s.score,s2.score)

Jamie
28
100 99


通常情况下，上面的set_score方法可以直接定义在class中，但动态绑定允许我们在程序运行的过程中动态给class加上功能，这在静态语言中很难实现。

# 使用\__slots__

当我们想要限制实例的属性怎么办？比如，只允许对Student实例添加name和age属性。为了达到限制的目的，Python允许在定义class的时候，**定义一个特殊的\__slots__变量，来限制该class实例能添加的属性**

由于'score'没有被放到\__slots__中，所以不能绑定score属性，试图绑定score将得到AttributeError的错误。

**注意\__slots__定义的属性仅对当前类实例起作用，对继承的子类是不起作用的**

除非在子类中也定义\__slots__，这样，**子类实例允许定义的属性就是自身的\__slots__加上父类的\__slots__**。

In [20]:
# 使用__slots__
class Student(object):
    __slots__ = ('name','age')
    
s = Student()
s.name = 'Jamie'
s.age = 28
# 由于'score'没有被放到__slots__中，所以不能绑定score属性，试图绑定score将得到AttributeError的错误。
#s.score = 100 # 报错 AttributeError: 'Student' object has no attribute 'score'


# slots定义的属性仅对当前类实例起作用，对继承的子类是不起作用的
class GraduateStudent(Student):
    pass
g = GraduateStudent()
g.score = 100
print(g.score)


#除非在子类中也定义__slots__，这样，子类实例允许定义的属性就是自身的__slots__加上父类的__slots__。

class Gstudent(Student):
    __slots__ = ('height')

gs = Gstudent()
gs.name = 'gs'
gs.age = 30
gs.height = 160
# gs.score =99 # AttributeError: 'Gstudent' object has no attribute 'score'

100


# 使用@property

为了限制score的范围，可以通过一个set_score()方法来设置成绩，再通过一个get_score()来获取成绩，这样，在set_score()方法里，就可以检查参数：

In [10]:
class Student(object):
    
    def get_score(self):
        return self._score
    
    def set_score(self,value):
        if not isinstance(value,int):
            raise ValueError('Score must be an interger!')
        if value < 0 or value > 100:
            raise ValueError('Score must be between 0~100')
            
        self._score = value
        

s = Student()
s.set_score(90)
s.get_score()
#s.set_score(999)

90

如果要让内部属性不被外部访问，可以把属性的名称前加上两个下划线\__，在Python中，实例的变量名如果以__开头，就变成了一个私有变量（private），只有内部可以访问，外部不能访问

有些时候，你会看到以一个下划线开头的实例变量名，比如_name，这样的实例变量外部是可以访问的，但是，按照约定俗成的规定，当你看到这样的变量时，意思就是，“虽然我可以被访问，但是，请把我视为私有变量，不要随意访问”。

但是，上面的调用方法又略显复杂，没有直接用属性这么直接简单。有没有既能检查参数，又可以用类似属性这样简单的方式来访问类的变量呢？还记得装饰器（decorator）可以给函数动态加上功能吗？对于类的方法，装饰器一样起作用。Python内置的@property装饰器就是负责把一个方法变成属性调用的：

In [18]:
class Student(object):
    
    @property
    def score(self):
        return self._score
    
    @score.setter
    def score(self,value):
        if not isinstance(value,int):
            raise ValueError('Score must be an interger!')
        if value < 0 or value > 100:
            raise ValueError('Score must be between 0~100')
            
        self._score = value
        

s = Student()
s.score = 60 # 实际事转化s.set_score(60)
s.score # 实际事转化s.get_score()

# s.score = 999

60

@property的实现比较复杂，我们先考察如何使用。把一个getter方法变成属性，只需要加上@property就可以了，此时，@property本身又创建了另一个装饰器@score.setter，负责把一个setter方法变成属性赋值，于是，我们就拥有一个可控的属性操作。 

注意到这个神奇的@property，我们在对实例属性操作的时候，就知道该属性很可能不是直接暴露的，而是通过getter和setter方法来实现的。

还可以定义只读属性，只定义getter方法，不定义setter方法就是一个只读属性：

In [21]:
class Student(object):
    
    @property
    def birth(self):
        return self._birth
    
    @birth.setter
    def birth(self,value):
        self._birth = value
    
    @property
    def age(self):
        return 2018-self._birth
    
s = Student()
s.birth = 28
s.age

1990

上面的birth是可读写属性，而age就是一个只读属性，因为age可以根据birth和当前时间计算出来。

# 定制类

看到类似\__slots__这种形如\__xxx__的变量或者函数名就要注意，这些在Python中是有特殊用途的。

\__slots__我们已经知道怎么用了，\__len__()方法我们也知道是为了能让class作用于len()函数。

除此之外，Python的class中还有许多这样有特殊用途的函数，可以帮助我们定制类。

## \__str__

用print打印实例的结果： `<__main__.Student object at 0x1107b4588>`，不好看。怎么才能打印得好看呢？只需要定义好__str__()方法，返回一个好看的字符串就可以了：

In [25]:
class Student():
    def __init__(self,name):
        self.name = name

s = Student('Jamie')
print(s)

print('__str__ 使 print 打印出定制内容')

class Student():
    def __init__(self,name):
        self.name = name
    def __str__(self):
        return 'Student object:{}'.format(self.name)

s = Student('Jamie')
print(s)

s

__str__ 使 print 打印出定制内容
Student object:Jamie


直接敲变量不用print，打印出来的实例还是不好看：`<__main__.Student at 0x110880f98>`

这是因为直接显示变量调用的不是\__str__()，而是\__repr__()，两者的区别是

- \__str__()返回用户看到的字符串

- \__repr__()返回程序开发者看到的字符串，也就是说，\__repr__()是为调试服务的。

解决办法是再定义一个\__repr__()。但是通常\__str__()和\__repr__()代码都是一样的，所以，有个偷懒的写法：

`__repr__ = __str__`

In [27]:
class Student():
    def __init__(self,name):
        self.name = name
    def __str__(self):
        return 'Student object:{}'.format(self.name)
    __repr__ =  __str__ 
    
s = Student('Jamie')
s

Student object:Jamie

## \__getattr__

正常情况下，当我们调用类的方法或属性时，如果不存在，就会报错。

In [28]:
class Student(object):
    def __init__(self,name):
        self.name = name
        
s = Student('Jamie')
print(s.name)
print(s.score)

Jamie


AttributeError: 'Student' object has no attribute 'score'

要避免这个错误，除了可以加上一个score属性外，Python还有另一个机制，那就是写一个__getattr__()方法，动态返回一个属性。

In [36]:
class Student(object):
    def __init__(self,name):
        self.name = name
    
    def __getattr__(self,attr):
        if attr == 'score':
            return 99
        if attr=='age':
            return lambda: 25
        
s = Student('Jamie')
print(s.name)
print(s.score)
print(s.age())
print(s.abe)

Jamie
99
25
None


当调用不存在的属性时，比如score，Python解释器会试图调用__getattr__(self, 'score')来尝试获得属性，这样，我们就有机会返回score的值：

返回函数也是完全可以的,只是调用方式要在函数后面加().

注意，只有在没有找到属性的情况下，才调用__getattr__，已有的属性，比如name，不会在__getattr__中查找。

此外，注意到任意调用如s.abc都会返回None，这是因为我们定义的__getattr__默认返回就是None。要让class只响应特定的几个属性，我们就要按照约定，抛出AttributeError的错误：



# 两句话掌握python最难知识点——元类

https://segmentfault.com/a/1190000011447445


学懂元类，你只需要知道两句话：

### 道生一，一生二，二生三，三生万物

### 我是谁？我从哪来里？我要到哪里去？


在python世界，拥有一个永恒的道，那就是“type”，请记在脑海中，type就是道。如此广袤无垠的python生态圈，都是由type产生出来的。

道生一，一生二，二生三，三生万物。

道 即是 type

一 即是 metaclass(元类，或者叫类生成器)

二 即是 class(类，或者叫实例生成器)

三 即是 instance(实例)

万物 即是 实例的各种属性与方法，我们平常使用python时，调用的就是它们。


In [1]:
# 创建一个Hello类，拥有属性 say_heelo --- 二的起源
class Hello():
    def say_hello(self,name='world'):
        print('Hello,{}'.format(name))

# 从 Hello 类创建一个实例 hello  --- 二生三
hello = Hello()

# 使用 hello 调用方法 say_hello --- 三生万物
hello.say_hello() 

print(type(Hello))
print(type(hello))

Hello,world
<class 'type'>
<class '__main__.Hello'>


这就是一个标准的“二生三，三生万物”过程。 从类到我们可以调用的方法，用了这两步。

那我们不由自主要问，类从何而来呢？回到代码的第一行。
class Hello其实是一个函数的“语义化简称”，只为了让代码更浅显易懂，它的另一个写法是：

这样的写法，就和之前的Class Hello写法作用完全相同,建实例并调用,输出效果也一样

In [13]:
# 假如我们有一个函数 fn
def fn(self,name='world'):
    print('Hello,{}'.format(name))

# 通过 type 创建 Hello 类   --- 道生二
Hello = type('Hello',(object,),dict(say_hello=fn))

hello = Hello()

hello.say_hello()

# 换成匿名函数
H = type('H',(object,),dict(say_hello=lambda self,value='world':print('Hello,{}'.format(value))))
h = H()
h.say_hello()

Hello,world
Hello,world


**`Hello = type('Hello', (object,), dict(say_hello=fn)) `**

**type(类名, 父类的元组（针对继承的情况，可以为空），包含属性的字典（名称和值）)**

注意它的三个参数！暗合人类的三大永恒命题：**我是谁，我从哪里来，我要到哪里去.**

第一个参数：我是谁。类名

第二个参数：我从哪里来。继承的父类集合，注意Python支持多重继承，如果只有一个父类，别忘了tuple的单元素写法；

第三个参数：我要到哪里去。需要调用的方法和属性包含到一个字典里，再作为参数传入。class的方法名称与函数绑定，这里我们把函数fn绑定到方法名hello上。


```
class Hello(object){
# class 后声明“我是谁”
# 小括号内声明“我来自哪里”
# 中括号内声明“我要到哪里去”
    def say_hello(){
        
    }
}
```



**以字典的模式给类定义方法或属性**

**属性：**

```
class Foo(object):
    bar = True
```
用type()创建有属性的类(类属性)
```
Foo = type('Foo',(object,),{'bar':Ture})
```

**方法：**

```
class Hello():
    def say_hello(self,name='world'):
        print('Hello,{}'.format(name))
```
用type()创建有方法的类(实例方法)
```
def fn(self,name='world'):
    print('Hello,{}'.format(name))

Hello = type('Hello',(object,),dict(say_hello=fn))
```
换匿名函数
```
Hello = type('Hello',(object,),dict(say_hello=lambda self,name='world':print('Hello,{}'.foramt(name))))
```



造物主，可以直接创造单个的人，但这是一件苦役。造物主会先创造“人”这一物种，再批量创造具体的个人。并将三大永恒命题，一直传递下去。

**“道”可以直接生出“二”，但它会先生出“一”，再批量地制造“二”。**

**type可以直接生成类（class），但也可以先生成元类（metaclass），再使用元类批量定制类（class）。**

## 元类——道生一，一生二

一般来说，元类均被命名后缀为Metalass。想象一下，我们需要一个可以自动打招呼的元类，它里面的类方法呢，有时需要say_Hello，有时需要say_Hi，有时又需要say_Sayolala，有时需要say_Nihao。

如果每个内置的say_xxx都需要在类里面声明一次，那将是多么可怕的苦役！ 不如使用元类来解决问题。

以下是创建一个专门“打招呼”用的元类代码：

In [ ]:
class SayMetaClass(type):
    
    def __new__(cls,name,bases,attrs):
        attrs['say_'+name]=lambda self,value,saying=name:print('{},{}!'.foramt(saying,value))
        return type.__new__(cls,name,bases,attrs)
    
    

记住两点：

1、元类是由“type”衍生而出，所以父类需要传入type。**【道生一，所以一必须包含道】**

2、元类的操作都在 \__new__中完成，它的第一个参数是将创建的类，之后的参数即是三大永恒命题：我是谁，我从哪里来，我将到哪里去。 它返回的对象也是三大永恒命题，接下来，这三个参数将一直陪伴我们。

在\__new__中，我只进行了一个操作，就是这个

`attrs['say_'+name] = lambda self,value,saying=name:print('{},{}!'.foramt(saying,value))`

其中匿名函数等同于
```
def niming(self,value,saying=name):
    print('{},{}!'.foramt(saying,value)

```

\__new__()方法接收到的参数依次是：

cls  表示元类

name  表示创建类的类名

bases 表示创建类继承的所有父类

attrs 表示创建类的所有属性和方法（以键值对的字典的形式）




## 根据道生一、一生二、二生三、三生万物的准则，走进元类的生命周期吧！

**注意：通过元类创建的类，第一个参数是父类，第二个参数是metaclass**

In [5]:
# 道生一 传入 type 
class SayMetaClass(type):
    # 传入三大永恒话题： 类名称、父类、属性
    def __new__(cls,name,bases,attrs):
        attrs['say_'+name] = lambda self,value,saying=name:print('{},{}!'.format(saying,value))
        # 传承三大永恒话题： 类名称、父类、属性
        return type.__new__(cls,name,bases,attrs)

# 一生二 创建类
class Hello(object,metaclass=SayMetaClass):
    pass

print('==============Hello================')

# 二生三 创建实例
hello = Hello()

# 三生万物 调用实例方法
hello.say_Hello('World')


print('==============Nihao================')

class Nihao(object,metaclass=SayMetaClass):
    pass

nihao = Nihao()
nihao.say_Nihao('中华')

print('===============Sayolala===============')

class Sayolala(object,metaclass=SayMetaClass):
    pass

sayolala = Sayolala()
sayolala.say_Sayolala('Japan')

==============Hello================
Hello,World!
==============Nihao================
Nihao,中华!
===============Sayolala===============
Sayolala,Japan!


如果不用元类，要达到上述效果，就需要创建3个类，里面各声明一次，如下：

In [16]:
print('==============Hello================')
class Hello(object):
    def say_Hello(self,value):
        print('Hello,{}!'.format(value))
        
hello = Hello()
hello.say_Hello('World')

print('==============Nihao================')

class Nihao(object):
    def say_Nihao(self,value):
        print('Nihao,{}!'.format(value))
        
nihao = Nihao()
nihao.say_Nihao('中华')

print('===============Sayolala===============')

class Sayolala(object):
    def say_Sayolala(self,value):
        print('Sayolala,{}!'.format(value))
        
sayolala = Sayolala()
sayolala.say_Sayolala('Japan')

==============Hello================
Hello,World!
==============Nihao================
Nihao,中华!
===============Sayolala===============
Sayolala,Japan!


在sayhello的基础上，加上问候方式，轻吻kiss，握手shakehand，鞠躬bow

In [20]:
# 道生一 传入 type 
class SayMetaClass(type):
    # 传入三大永恒话题： 类名称、父类、属性
    def __new__(cls,name,bases,attrs):
        attrs['say_'+name] = lambda self,value,saying=name:print('{},{}!'.format(saying,value))
        attrs['manner'] = lambda self,manner:print('{}！'.format(manner))
        # 传承三大永恒话题： 类名称、父类、属性
        return type.__new__(cls,name,bases,attrs)

# 一生二 创建类
class Hello(object,metaclass=SayMetaClass):
    pass

print('==============Hello================')

# 二生三 创建实例
hello = Hello()

# 三生万物 调用实例方法
hello.say_Hello('World')
hello.manner('Kiss')


print('==============Nihao================')

class Nihao(object,metaclass=SayMetaClass):
    pass

nihao = Nihao()
nihao.say_Nihao('中华')
nihao.manner('Shakehand')

print('===============Sayolala===============')

class Sayolala(object,metaclass=SayMetaClass):
    pass

sayolala = Sayolala()
sayolala.say_Sayolala('Japan')
sayolala.manner('Bow')

==============Hello================
Hello,World!
Kiss！
==============Nihao================
Nihao,中华!
Shakehand！
===============Sayolala===============
Sayolala,Japan!
Bow！


##  给list增加一个list本身没有的add方法

In [23]:
class ListMetaClass(type):
    def __new__(cls,name,bases,attrs):
        attrs['add'] = lambda self,value:self.append(value)
        return type.__new__(cls,name,bases,attrs)

class MyList(list,metaclass=ListMetaClass):
    pass

L = MyList()
L.add(1)
print(L)

L2 = list()
L2.add(1)

[1]


AttributeError: 'list' object has no attribute 'add'

动态修改有什么意义？直接在MyList定义中写上add()方法不是更简单吗？正常情况下，确实应该直接写，通过metaclass修改纯属变态。但是，总会遇到需要通过metaclass修改类定义的。ORM就是一个典型的例子。

## 通过元类创建 ORM

https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/0014319106919344c4ef8b1e04c48778bb45796e0335839000

ORM全称“Object Relational Mapping”，即对象-关系映射，就是把关系数据库的一行映射为一个对象，也就是一个类对应一个表，这样，写代码更简单，不用直接操作SQL语句。

让我们来尝试编写一个ORM框架。

下面代码中使用到的 super()，可参考 [Super使用](#jump)

In [22]:
# 创建一个 Field 类

class Field(object):
    def __init__(self,name,column_type):
        self.name = name
        self.column_type = column_type
        
    def __str__(self):
        return '{}:{}'.format(self.__class__.__name__,self.name)
'''
它的作用是
在Field类实例化时将得到两个参数，name和column_type，它们将被绑定为Field的私有属性，
如果要将Field转化为字符串时，将返回“Field:XXX” ， XXX是传入的name名称。
'''

# 创建 StringField 和 IntegerField

class StringField(Field):
    def __init__(self,name):
        super().__init__(name,'varchar(100)')
    
class IntegerField(Field):
    def __init__(self,name):
        super().__init__(name,'bigint')

dataa = StringField('数据a')
print(dataa)
        
        
'''
它的作用是
在StringField,IntegerField实例初始化时，时自动调用父类的初始化方式。
'''

# 道生一：传入type

'''
class SayMetaClass(type):

    # 传入三大永恒命题：类名称、父类、属性
    def __new__(cls, name, bases, attrs):
        # 创造“天赋”
        attrs['say_'+name] = lambda self,value,saying=name: print(saying+','+value+'!')
        # 传承三大永恒命题：类名称、父类、属性
        return type.__new__(cls, name, bases, attrs)
'''

class ModelMetaclass(type):
    
    def __new__(cls,name,bases,attrs):
        if name == 'Model':
            return type.__new__(cls,name,bases,attrs)
        print('Build Model:{}'.foramt(name))
        mappings = dict()
        for k,v in attrs.items():
            if isinstance(v,Field):
                print('Build mapping:{}->{}'.format(k,v))
                mappings[k]=v
        for k in mappings.keys():
            attrs.pop(k)
        attrs['__mappings__'] = mappings
        attrs['__table__'] = name
        return type.__new__(cls,name,bases,attrs)
    

class Model(object,metaclass=ModelMetaclass):
    a = 1
    b = '2'

m = Model()
print(m.mappings)

StringField:数据a


AttributeError: 'Model' object has no attribute 'mappings'

# python3 下的元类窥探及实现

https://blog.csdn.net/cz19900520/article/details/79641597



# <span id="jump">Super() 使用</span>

http://python.jobbole.com/86787/

在类的继承中，如果重定义某个方法，该方法会覆盖父类的同名方法，但有时，我们希望能同时实现父类的功能，这时，我们就需要调用父类的方法了，可通过使用 super 来实现，比如：

In [18]:
class Animal(object):
    def __init__(self,name):
        self.name = name
    def greet(self):
        print('Hello,{}'.format(self.name))
        
class Dog(Animal):
    def greet(self):
        super().greet() # Python3 super(Dog, self).greet()
        print('WangWang...')
    
dog = Dog('dog')
dog.greet()

class D(object):
    def test(self):
        print('test in D')

class C(D):
    def test(self):
        print('test in C')
        D.test(self)

    c = C()
c.test()


Hello,dog
WangWang...
test in C
test in D


在上面，Animal 是父类，Dog 是子类，我们在 Dog 类重定义了 greet 方法，为了能同时实现父类的功能，我们又调用了父类的方法。

为了同时拥有父类的属性，super 的一个最常见用法可以说是在子类中调用父类的初始化方法了。

In [5]:
class Base(object):
    def __init__(self,a,b):
        self.a = a
        self.b = b
        print('Base:a{},b{}'.format(self.a,self.b))
        
class A(Base):
    def __init__(self,a,b,c):
        super().__init__(a,b)  # Python2 super(A, self).__init__(a, b)
        self.c = c
        print('A:a{},b{},c{}'.format(self.a,self.b,self.c))

test = A(1,2,3)

Base:a1,b2
A:a1,b2,c3


## 深入 super()

看了上面的使用，你可能会觉得 super 的使用很简单，无非就是获取了父类，并调用父类的方法。其实，在上面的情况下，super 获得的类刚好是父类，但在其他情况就不一定了，super 其实和父类没有实质性的关联。

让我们看一个稍微复杂的例子，涉及到多重继承，代码如下：

In [10]:
class Base(object):
    def __init__(self):
        print('enter Base')
        print('leave Base')
        
class A(Base):
    def __init__(self):
        print('enter A')
        super().__init__()
        print('leave A')
        
class B(Base):
    def __init__(self):
        print('enter B')
        super().__init__()
        print('leave B')
        
        
class C(A,B):
    def __init__(self):
        print('enter C')
        super().__init__()
        print('leave C')
        
a = A()

print('---------------------')
b = B()

print('---------------------')
c = C()
print('---------------------')
C.mro()

enter A
enter Base
leave Base
leave A
---------------------
enter B
enter Base
leave Base
leave B
---------------------
enter C
enter A
enter B
enter Base
leave Base
leave B
leave A
leave C
---------------------


[__main__.C, __main__.A, __main__.B, __main__.Base, object]

c = C() 预测的结果是

```
enter C
enter A
enter Base
leave Base
leave A
enter B
enter Base
leave Base
leave B
leave C

```

如果你认为 super 代表『调用父类的方法』，那你很可能会疑惑为什么 enter A 的下一句不是 enter Base 而是 enter B。原因是，**super 和父类没有实质性的关联**，现在让我们搞清 super 是怎么运作的。

## MRO 列表 (类)

事实上，对于你定义的每一个类，Python 会计算出一个**s方法解析顺序（Method Resolution Order, MRO）列表**，它代表了类继承的顺序，我们可以使用下面的方式获得某个类的 MRO 列表：

```
>>> C.mro()   # or C.__mro__ or C().__class__.mro()
[__main__.C, __main__.A, __main__.B, __main__.Base, object]

```

那这个 MRO 列表的顺序是怎么定的呢，它是通过一个 C3 线性化算法来实现的，这里我们就不去深究这个算法了，感兴趣的读者可以自己去了解一下，总的来说，一个类的 MRO 列表就是合并所有父类的 MRO 列表，并遵循以下三条原则：

- 子类永远在父类前面

- 如果有多个父类，会根据它们在列表中的顺序被检查

- 如果对下一个类存在两个合法的选择，选择第一个父类

## super 原理

为了更好演示，上述代码用 Python2 的格式

```
def super(cls,inst):
    mro = inst.__class__.mro()
    return mro[mro.index(cls)+1]
```
其中，cls 代表类，inst 代表实例，上面的代码做了两件事：

- 获取 inst 的 MRO 列表

- 查找 cls 在当前 MRO 列表中的 index, 并返回它的下一个类，即 mro[index + 1]

当你使用 super(cls, inst) 时，Python 会在 inst 的 MRO 列表上搜索 cls 的下一个类。

In [4]:
class Base(object):
    def __init__(self):
        print("enter Base")
        print("leave Base")

class A(Base):
    def __init__(self):
        print ("enter A")
        super(A, self).__init__()
        print("leave A")

class B(Base):
    def __init__(self):
        print("enter B")
        super(B, self).__init__()
        print("leave B")
        
class C(A, B):
    def __init__(self):
        print("enter C")
        super(C, self).__init__()
        print("leave C")

c = C()
C.mro()

enter C
enter A
enter B
enter Base
leave Base
leave B
leave A
leave C


[__main__.C, __main__.A, __main__.B, __main__.Base, object]

首先看类 C 的 __init__ 方法：`super(C, self).__init__()`

根据super()函数，
```
def super(cls,inst):
    mro = inst.__class__.mro()
    return mro[mro.index(cls)+1]
```
其中`super(C, self)`会返回 C.mro() 列表中后面一位，即`__main__.A`, 就变成`类A的 __init__()`，这时打印出 enter A，并执行下一行 `super(A, self).__init__()`

**注意：这里的 self 也是当前 C 的实例，MRO 列表跟上面是一样的**

搜索 A 在 MRO 中的下一个类，发现是 B，于是，跳到了`类B的__init__()`，这时会打印出 enter B，而不是 enter Base。并执行下一行 `super(B, self).__init__()`

同样，下一个类是 Base，跳到了`类Base的__init__()`，打印 enter Base 和 leave Base。

`类Base.__init__()`执行完，跳回到类B中的，执行`super(B, self).__init__()`下面一句，即打印 leave B。

`类B的__init__()`执行完，跳回到类A中的，执行`super(A, self).__init__()`下面一句，即打印 leave A。

`类A的__init__()`执行完，跳回到类C中的，执行`super(C, self).__init__()`下面一句，即打印 leave C。

整个过程还是比较清晰的，关键是要理解 super 的工作方式，而不是想当然地认为 super 调用了父类的方法。

## 小结

- 事实上，super 和父类没有实质性的关联。

- super(cls, inst) 获得的是 cls 在 inst 的 MRO 列表中的下一个类。